# TensorFlow Modeling for [LEGO Minifigures Classification](https://www.kaggle.com/ihelon/lego-minifigures-classification) dataset

This is the guide about using pre-trained models in TensorFlow and Keras frameworks.   
We will use the MobileNetV2 model to predict which Minifigure is in the image.   

![](https://i.imgur.com/4cPQlEN.jpg)

<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='background:blue; border:0; color:white' role="tab" aria-controls="home"><center>Quick Navigation</center></h3>

* [1. Configurations](#1)
* [2. Data reading](#2)
* [3. Data generator](#3)
* [4. Augmentations](#4)
* [5. Train and valid generators](#5)
* [6. Data visualizations (train samples)](#6)   
* [7. Data visualizations (valid samples)](#7)   
* [8. Model initialization](#8)   
* [9. Checkpoints initialization](#9)
* [10. Model training](#10)
* [11. Train logs](#11)
* [12. Final test score check](#12)
* [13. Error analysis - Confusion matrix](#13)
* [14. Error analysis - Misclassified samples](#14)

<a id="1"></a>
<h2 style='background:blue; border:0; color:white'><center>Configurations<center><h2>

In [ ]:
import os
import math
import random

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sn
import albumentations as A
import tensorflow as tf
from tensorflow.keras.applications import mobilenet_v2 as tf_mobilenet_v2
from tensorflow.keras import layers as tf_layers
from tensorflow.keras import models as tf_models
from tensorflow.keras import callbacks as tf_callbacks
from sklearn import metrics as sk_metrics
from sklearn import model_selection as sk_model_selection

The directory to the dataset

In [ ]:
BASE_DIR = "../input/lego-minifigures-classification/"
PATH_INDEX = os.path.join(BASE_DIR, "index.csv")
PATH_TEST = os.path.join(BASE_DIR, "test.csv")
PATH_METADATA = os.path.join(BASE_DIR, "metadata.csv")

Let's define all out configs and parameters in the one place

In [ ]:
config = {
    "seed": 42,
    
    "valid_size": 0.3,
    
    "image_size": (512, 512),
    "train_batch_size": 4,
    "valid_batch_size": 1,
    "test_batch_size": 1,
    
    "model": "mobilenet_v2",
    "max_epochs": 50,
    "patience_stop": 3,
    "path_to_save_model": "best.hdf5",
    "callbacks_monitor": "val_loss",
    
}

Try to set random seet that our experiment repeated between (We have some problem to set seed with GPU in Kaggle)

In [ ]:
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
    os.environ["PYTHONHASHSEED"] = str(seed_value)
    os.environ["TF_DETERMINISTIC_OPS"] = "true"
    

set_seed(config["seed"])

<a id="2"></a>
<h2 style='background:blue; border:0; color:white'><center>Data reading<center><h2>

In [ ]:
# Read information about dataset
df = pd.read_csv(PATH_INDEX)

tmp_train, tmp_valid = sk_model_selection.train_test_split(
    df, 
    test_size=config["valid_size"], 
    random_state=config["seed"], 
    stratify=df['class_id'],
)


def get_paths_and_targets(tmp_df):
    # Get file paths
    paths = tmp_df["path"].values
    # Create full paths (base dir + concrete file name)
    paths = list(map(lambda x: os.path.join(BASE_DIR, x), paths))
    # Get labels
    targets = tmp_df["class_id"].values
    
    return paths, targets


# Get train file paths and targets
train_paths, train_targets = get_paths_and_targets(tmp_train)

# Get valid file paths and targets
valid_paths, valid_targets = get_paths_and_targets(tmp_valid)

df_test = pd.read_csv(PATH_TEST)
# Get test file paths and targets
test_paths, test_targets = get_paths_and_targets(df_test)

In [ ]:
# Total number of classes in the dataset
df_metadata = pd.read_csv(PATH_METADATA)
n_classes = df_metadata.shape[0]
print("Number of classes: ", n_classes)

<a id="3"></a>
<h2 style='background:blue; border:0; color:white'><center>Data generator<center><h2>

DataGenerator allows you not to load the entire dataset to memory at once, but to do it in batches     
Each time we have only one batch of pictures in memory

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(
        self, paths, targets, image_size=(224, 224), batch_size=64, 
        shuffle=True, transforms=None, preprocess=None,
    ):
        # the list of paths to files
        self.paths = paths
        # the list with the true labels of each file
        self.targets = targets
        # images size
        self.image_size = image_size
        # batch size (the number of images)
        self.batch_size = batch_size
        # if we need to shuffle order of files
        # for validation we don't need to shuffle, for training - do
        self.shuffle = shuffle
        # Augmentations for our images. It is implemented with albumentations library
        self.transforms = transforms
        # Preprocess function for the pretrained model. 
        # CHANGE IT IF USING OTHER THAN MOBILENETV2 MODEL
        self.preprocess = preprocess
        
        # Call function to create and shuffle (if needed) indices of files
        self.on_epoch_end()
        
    def on_epoch_end(self):
        # This function is called at the end of each epoch while training
        
        # Create as many indices as many files we have
        self.indexes = np.arange(len(self.paths))
        # Shuffle them if needed
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __len__(self):
        # We need that this function returns the number of steps in one epoch
        
        # How many batches we have
        return len(self.paths) // self.batch_size
    
    
    def __getitem__(self, index):
        # This function returns batch of pictures with their labels
        
        # Take in order as many indices as our batch size is
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        
        # Take image file paths that are included in that batch
        batch_paths = [self.paths[k] for k in indexes]
        # Take labels for each image
        batch_y = [self.targets[k] - 1 for k in indexes]
        batch_X = []
        for i in range(self.batch_size):
            # Read the image
            img = cv2.imread(batch_paths[i])
            # Resize it to needed shape
            img = cv2.resize(img, self.image_size)
            # Convert image colors from BGR to RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Apply transforms (see albumentations library)
            if self.transforms:
                img = self.transforms(image=img)["image"]
            # Apply preprocess
            if self.preprocess:
                img = self.preprocess(img)
            
            batch_X.append(img)
            
        return np.array(batch_X), np.array(batch_y)


<a id="4"></a>
<h2 style='background:blue; border:0; color:white'><center>Augmentations<center><h2>

Albumentations augmentations for the train data.       
We don't need this transformations for the validation.   
[albumentations-demo](https://albumentations-demo.herokuapp.com/) 

In [ ]:
def get_train_transforms():
    return A.Compose(
        [
            A.ShiftScaleRotate(
                p=1.0, 
                shift_limit=(-0.1, 0.1), 
                scale_limit=(-0.2, 0.2), 
                rotate_limit=(-30, 30), 
                border_mode=4
            ),
            A.CoarseDropout(
                p=0.5, 
                max_holes=100, 
                max_height=50, 
                max_width=50, 
                min_holes=10, 
                min_height=10, 
                min_width=10,
                fill_value=0,
            ),
            A.CoarseDropout(
                p=0.5, 
                max_holes=100, 
                max_height=50, 
                max_width=50, 
                min_holes=10, 
                min_height=10, 
                min_width=10,
                fill_value=255,
            ),
            A.HorizontalFlip(p=0.5),
            A.RandomContrast(limit=(-0.3, 0.3), p=0.5),
            A.RandomBrightness(limit=(-0.4, 0.4), p=0.5),
            A.Blur(p=0.25),
        ], 
        p=1.0
    )

<a id="5"></a>
<h2 style='background:blue; border:0; color:white'><center>Train and valid generators<center><h2>

In [ ]:
# Initialize the train data generator
train_generator = DataGenerator(
    train_paths, 
    train_targets, 
    batch_size=config["train_batch_size"], 
    image_size=config["image_size"],
    shuffle=True, 
    transforms=get_train_transforms(),
    preprocess=tf_mobilenet_v2.preprocess_input,
)

# Initialize the valid data generator
valid_generator = DataGenerator(
    valid_paths, 
    valid_targets, 
    image_size=config["image_size"],
    batch_size=config["valid_batch_size"], 
    shuffle=False,
    preprocess=tf_mobilenet_v2.preprocess_input,
)

# Initialize the test data generator
test_generator = DataGenerator(
    test_paths, 
    test_targets, 
    image_size=config["image_size"],
    batch_size=config["test_batch_size"], 
    shuffle=False,
    preprocess=tf_mobilenet_v2.preprocess_input,
)

<a id="6"></a>
<h2 style='background:blue; border:0; color:white'><center>Data visualizations (train samples)<center><h2>

In [ ]:
def denormalize_image(image):
    return ((image + 1) * 127.5).astype(int)

# Let's visualize some batches of the train data
plt.figure(figsize=(16, 16))
ind = 0
for i_batch in range(len(train_generator)):
    images, labels = train_generator[i_batch]
    for i in range(len(images)):
        plt.subplot(5, 5, ind + 1)
        ind += 1
        plt.imshow(denormalize_image(images[i]))
        plt.title(f"class: {labels[i]}")
        plt.axis("off")
        if ind >= 25:
            break
    if ind >= 25:
        break

<a id="7"></a>
<h2 style='background:blue; border:0; color:white'><center>Data visualizations (valid samples)<center><h2>

In [ ]:
plt.figure(figsize=(16, 16))
ind = 0
for i_batch in range(len(valid_generator)):
    images, labels = valid_generator[i_batch]
    for i in range(len(images)):
        plt.subplot(5, 5, ind + 1)
        ind += 1
        plt.imshow(denormalize_image(images[i]))
        plt.title(f"class: {labels[i]}")
        plt.axis("off")
        if ind >= 25:
            break
    if ind >= 25:
        break

<a id="8"></a>
<h2 style='background:blue; border:0; color:white'><center>Model initialization<center><h2>

We use simple MobileNetV2 model as a backbone

In [ ]:
def init_mobilenet_v2(n_classes):
    # We take pretrained MobileNetV2 (see Keras docs)
    base_model = tf_mobilenet_v2.MobileNetV2()
    x = base_model.layers[-2].output
    # Take penultimate layer of the MobileNetV2 model and connect this layer with Dropout
    x = tf_layers.Dropout(.5)(x)
    # Add additional Dense layer, with number of neurons as number of our classes
    # Use softmax activation because we have one class classification problem
    outputs = tf_layers.Dense(n_classes, activation="softmax")(x)
    # Create model using MobileNetV2 input and our created output
    model = tf_models.Model(base_model.inputs, outputs)
    
    return model


def compile_model(model, optimizer, loss, metrics):
    # Compile model using Adam optimizer and categorical crossentropy loss
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model


models_constructor = {
    "mobilenet_v2": init_mobilenet_v2,
}

<a id="9"></a>
<h2 style='background:blue; border:0; color:white'><center>Checkpoints initialization<center><h2>

In [ ]:
# checkpoint to saving the best model by validation loss
callback_save = tf_callbacks.ModelCheckpoint(
    config["path_to_save_model"],
    monitor=config["callbacks_monitor"],
    save_best_only=True,
)

# checkpoint to stop training if model didn't improve valid loss for 3 epochs
callback_early_stopping = tf_callbacks.EarlyStopping(
    monitor=config["callbacks_monitor"],
    patience=config["patience_stop"],
)

<a id="10"></a>
<h2 style='background:blue; border:0; color:white'><center>Model training<center><h2>

In [ ]:
model = models_constructor[config["model"]](n_classes)
compile_model(
    model,
    tf.keras.optimizers.Adam(0.0001), 
    "sparse_categorical_crossentropy", 
    ["accuracy"],
)

# Train model using data generators
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=config["max_epochs"],
    callbacks=[callback_save, callback_early_stopping],
    verbose=1,
)

<a id="11"></a>
<h2 style='background:blue; border:0; color:white'><center>Train logs<center><h2>

In [ ]:
def plot_history(param="loss"):
    plt.figure(figsize=(6, 6))
    if param == "loss":
        plt.plot(history.history["loss"], label="train loss")
        plt.plot(history.history["val_loss"], label="valid loss")
        plt.ylabel("Loss value", fontsize=15)
    elif param == "accuracy":
        plt.plot(history.history["accuracy"], label="train acc")
        plt.plot(history.history["val_accuracy"], label="valid acc")
        plt.ylim(0, 1)
        plt.ylabel("Accuracy score", fontsize=15)
    plt.xticks(fontsize=14)
    plt.xlabel("Epoch number", fontsize=15)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=15)
    plt.grid()
    plt.show()


# Visualize train and valid loss 
plot_history("loss")
# Visualize train and valid accyracy 
plot_history("accuracy")

<a id="12"></a>
<h2 style='background:blue; border:0; color:white'><center>Final test check<center><h2>

In [ ]:
# Load the best model (we create for checkpoint to save the best model)
model = tf_models.load_model(config["path_to_save_model"])

In [ ]:
# Save the model predictions and true labels
y_pred = []
y_test = []
for _X_test, _y_test in test_generator:
    y_pred.extend(model.predict(_X_test).argmax(axis=-1))
    y_test.extend(_y_test)

# Calculate needed metrics
print(f"Accuracy score on test data: {sk_metrics.accuracy_score(y_test, y_pred)}")
print(f"Macro F1 score on test data: {sk_metrics.f1_score(y_test, y_pred, average='macro')}")

<a id="13"></a>
<h2 style='background:blue; border:0; color:white'><center>Error analysis - Confusion matrix<center><h2>

In [ ]:
# Load metadata to get classes people-friendly names
labels = df_metadata["minifigure_name"].tolist()

# Calculate confusion matrix
confusion_matrix = sk_metrics.confusion_matrix(y_test, y_pred)
df_confusion_matrix = pd.DataFrame(confusion_matrix, index=labels, columns=labels)

# Show confusion matrix
plt.figure(figsize=(12, 12))
sn.heatmap(df_confusion_matrix, annot=True, cbar=False, cmap="Oranges", linewidths=1, linecolor="black")
plt.xlabel("Predicted labels", fontsize=15)
plt.xticks(fontsize=12)
plt.ylabel("True labels", fontsize=15)
plt.yticks(fontsize=12);

<a id="14"></a>
<h2 style='background:blue; border:0; color:white'><center>Error analysis - Misclassified samples<center><h2>

In [ ]:
# Save image, label, prediction for false predictions 
error_images = []
error_label = []
error_pred = []
error_prob = []
for _X_test, _y_test in test_generator:
    pred = model.predict(_X_test).argmax(axis=-1)
    if pred[0] != _y_test:
        error_images.extend(_X_test)
        error_label.extend(_y_test)
        error_pred.extend(pred)
        error_prob.extend(model.predict(_X_test).max(axis=-1))

In [ ]:
# Visualize missclassified samples
w_size = 3
h_size = math.ceil(len(error_images) / w_size)
plt.figure(figsize=(16, h_size * 4))
for ind, image in enumerate(error_images):
    plt.subplot(h_size, w_size, ind + 1)
    plt.imshow(denormalize_image(image))
    pred_label = labels[error_pred[ind]]
    pred_prob = error_prob[ind]
    true_label = labels[error_label[ind]]
    plt.title(f"predict: {pred_label} ({pred_prob:.2f})\ntrue: {true_label}", fontsize=12)
    plt.axis("off")